# Tutorial 4 - Memory

## Key Philosophy
- It would be important to learn from past experience and improve the agentic framework - memory is key to that
- You can add to the memory bank of your Agents pre-inference (by collecting from a pool of data prior to running the Agent), or during inference (add on in between running subtasks)

## Use Memory in Agents
- Agent class takes `memory_bank` as a parameter during initialisation of an `Agent`
- memory_bank: class Dict[Memory]. Stores multiple types of memory for use by the agent. Customise the Memory config within the Memory class.
    - Default: `memory_bank = {'Function': Memory(top_k = 5, mapper = lambda x: x.fn_description, approach = 'retrieve_by_ranker')}`
    - Key: `Function` (Already Implemented Natively) - Does RAG over Task -> Function mapping
    - Can add in more keys that would fit your use case. Retrieves similar items to task/overall plan (if able) for additional context in `get_next_subtasks()` and `use_llm()` function
    - Side Note: RAG can also be done (and may be preferred) as a separate function of the Agent to retrieve more information when needed (so that we do not overload the Agent with information)

## Memory Class
- Retrieves top k memory items based on task 
- Inputs:
    - `memory`: List. Default: Empty List. The list containing the memory items
    - `top_k`: Int. Default: 3. The number of memory list items to retrieve
    - `mapper`: Function. Maps the memory item to another form for comparison by ranker or LLM. Default: `lambda x: x`
        - Example mapping: `lambda x: x.fn_description` (If x is a Class and the string you want to compare for similarity is the fn_description attribute of that class)
    - `approach`: str. Either `retrieve_by_ranker` or `retrieve_by_llm` to retrieve memory items.
        - Ranker is faster and cheaper as it compares via embeddings, but are inferior to LLM-based methods for contextual information
    - `ranker`: `Ranker`. The Ranker which defines a similarity score between a query and a key. Default: OpenAI `text-embedding-3-small` model. 
        - Can be replaced with a function which returns similarity score from 0 to 1 when given a query and key
        
## Example Use Case
- Helps to reduce number of functions present in LLM context for more accurate generation
```python
output = my_agent.run('Calculate 2**10 * (5 + 1) / 10')
```

`Original Function List: add_numbers, subtract_numbers, add_three_numbers, multiply_numbers, divide_numbers, power_of, GCD_of_two_numbers, modulo_of_numbers, absolute_difference, generate_poem_with_numbers, List_related_words, generate_quote`

`Filtered Function Names: add_three_numbers, multiply_numbers, divide_numbers, power_of, modulo_of_numbers`

In [1]:
# !pip install taskgen-ai

In [2]:
# Set up API key and do the necessary imports
import os
from taskgen import *

os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

# Use Case 1: Filtering Functions by Task

In [3]:
import math
# Define multiple functions
fn_list = [
    Function("Adds <num1> to <num2>", output_format = {"Output": "num1 + num2"}, external_fn = lambda num1, num2: num1 + num2),
    Function("Subtracts <num1> from <num2>", output_format = {"Output": "num1 - num2"}, external_fn = lambda num1, num2: num1 + num2),
    Function("Adds <num1>, <num2> and <num3>", output_format = {"Output": "num1 + num2 + num3"}, external_fn = lambda num1, num2, num3: num1 + num2 + num3),
    Function("Multiply <num1> by <num2>", output_format = {"Output": "num1 * num2"}, external_fn = lambda num1, num2: num1 * num2),
    Function("Divide <num1> by <num2>", output_format = {"Output": "num1 / num2"}, external_fn = lambda num1, num2: num1 / num2),
    Function("Returns <num1>**<num2>", output_format = {"Output": "num1**num2"}, external_fn = lambda num1, num2: math.pow(num1,num2)),
    Function("Returns Greatest Common Divisor of <num1> and <num2>", output_format = {"Output": "GCD(num1, num2)"}, external_fn = lambda num1, num2: math.gcd(num1, num2)),
    Function("Returns modulo of <num1> over <num2>", output_format = {"Output": "num1 % num2"}, external_fn = lambda num1, num2: num1 % num2),
    Function("Returns absolute difference between <num1> and <num2>", output_format = {"Output": "abs(num1 - num2)"}, external_fn = lambda num1, num2: math.abs(num1-num2)),
    Function("Generates a poem containing <num1> and <num2>", output_format = {"Output": "Poem"}),
    Function("Lists out <num> words related to <word>", output_format = {"Output": "List of words, type: list"}),
    Function("Generates a quote about <topic>", output_format = {"Output": "Quote"})
]

In [4]:
my_agent = Agent('Generalist Agent', 'Does everything').assign_functions(fn_list)

In [5]:
# see the auto-generated names of your functions :)
my_agent.list_functions()

["Name: use_llm\nDescription: Used only when no other function can do the task\nInput: []\nOutput: {'Output': 'Output of LLM'}\n",
 'Name: end_task\nDescription: Use only when task is completed\nInput: []\nOutput: {}\n',
 "Name: add_numbers\nDescription: Adds <num1> to <num2>\nInput: ['num1', 'num2']\nOutput: {'Output': 'num1 + num2'}\n",
 "Name: subtract_numbers\nDescription: Subtracts <num1> from <num2>\nInput: ['num1', 'num2']\nOutput: {'Output': 'num1 - num2'}\n",
 "Name: add_three_numbers\nDescription: Adds <num1>, <num2> and <num3>\nInput: ['num1', 'num2', 'num3']\nOutput: {'Output': 'num1 + num2 + num3'}\n",
 "Name: multiply_numbers\nDescription: Multiply <num1> by <num2>\nInput: ['num1', 'num2']\nOutput: {'Output': 'num1 * num2'}\n",
 "Name: divide_numbers\nDescription: Divide <num1> by <num2>\nInput: ['num1', 'num2']\nOutput: {'Output': 'num1 / num2'}\n",
 "Name: power\nDescription: Returns <num1>**<num2>\nInput: ['num1', 'num2']\nOutput: {'Output': 'num1**num2'}\n",
 "Name: g

In [6]:
# visualise how the Functions are chosen based on task - here you see subtract_numbers appearing at the front
[f.fn_name for f in my_agent.memory_bank['Function'].retrieve_by_ranker('subtract numbers')]

['subtract_numbers',
 'add_three_numbers',
 'add_numbers',
 'divide_numbers',
 'multiply_numbers']

In [7]:
my_agent.reset()
output = my_agent.run('Calculate 2**10 * (5 + 1) / 10')

Filtered Function Names: add_three_numbers, multiply_numbers, divide_numbers, power, modulo_of_numbers
Subtask identified: Calculate 2**10
Calling function power with parameters {'num1': 2, 'num2': 10}
> 1024.0

Filtered Function Names: add_three_numbers, multiply_numbers, divide_numbers, greatest_common_divisor, modulo_of_numbers
Subtask identified: Add 5 + 1
Calling function add_three_numbers with parameters {'num1': 5, 'num2': 1, 'num3': 0}
> 6

Filtered Function Names: add_three_numbers, multiply_numbers, divide_numbers, greatest_common_divisor, modulo_of_numbers
Subtask identified: Multiply the result by 2**10
Calling function multiply_numbers with parameters {'num1': 6, 'num2': 1024}
> 6144

Filtered Function Names: add_three_numbers, multiply_numbers, divide_numbers, greatest_common_divisor, modulo_of_numbers
Subtask identified: Divide the final result by 10
Calling function divide_numbers with parameters {'num1': 6144, 'num2': 10}
> 614.4

Filtered Function Names: add_three_num

In [8]:
my_agent.status()

Agent Name: Generalist Agent
Agent Description: Does everything
Available Functions: ['use_llm', 'end_task', 'add_numbers', 'subtract_numbers', 'add_three_numbers', 'multiply_numbers', 'divide_numbers', 'power', 'greatest_common_divisor', 'modulo_of_numbers', 'absolute_difference', 'generate_poem_with_numbers', 'list_related_words', 'generate_quote']
Task: Calculate 2**10 * (5 + 1) / 10
Subtasks Completed:
Subtask: Calculate 2**10
1024.0

Subtask: Add 5 + 1
6

Subtask: Multiply the result by 2**10
6144

Subtask: Divide the final result by 10
614.4

Is Task Completed: True


# Use Case 2: Adding more context based on task

In [9]:
# first append more context to the agent
my_agent.memory_bank['Number Meanings'] = Memory(['Azo means 1', 'Boneti means 2', 'Andkh means 3', 'Bdakf means 4', 'dafdsk means 5', 
                                        'ldsfn means 6', 'sdkfn means 7', 'eri means 8', 'knewro means 9', 'mdsnfk means 10'], # some nonsense words
                                                top_k = 5,  # choose top 5
                                                mapper = lambda x: x.split(' ')[0]) # we compare with the task using only the first word, e.g. Azo, Boneti, Andkh

In [10]:
my_agent.reset()
output = my_agent.run('Evaluate Boneti + mdsnfk + Azo. Substitute words into numbers first before calling a numerical function')

Filtered Function Names: add_numbers, add_three_numbers, multiply_numbers, modulo_of_numbers, generate_poem_with_numbers
Subtask identified: Substitute words into numbers for Boneti, mdsnfk, and Azo
Getting LLM to perform the following task: Substitute words into numbers for Boneti, mdsnfk, and Azo
> Evaluate 2 + 10 + 1 = 13

Filtered Function Names: add_numbers, subtract_numbers, add_three_numbers, multiply_numbers, generate_poem_with_numbers
Subtask identified: Add the numerical values together
Calling function add_three_numbers with parameters {'num1': 2, 'num2': 10, 'num3': 1}
> 13

Filtered Function Names: add_numbers, subtract_numbers, add_three_numbers, multiply_numbers, generate_poem_with_numbers
Task completed successfully!



# Comparison between embedding-based methods using Ranker and LLM-based similarity
- Pros of embedding-based similarity comparison: Fast and cheap
- Cons of embedding-based similarity comparison: Not as accurate
- If using default `Ranker` function (OpenAI embedding model), automatically stores new embeddings generated in `database` and uses back known embeddings from `database` when possible, potentially helping to save time and costs
- Select the right method for your use case

- (Advanced Exercise) Instead of using cosine similarity using OpenAI Embeddings, create your own `ranking_fn` within `Ranker` that does similarity search the way you want it to

In [11]:
database = {}
memory = Memory(['hello', 'no worries', 'goodbye', 'hurray'], top_k = 1, ranker = Ranker(database = database))
print('Using embeddings', memory.retrieve_by_ranker('Another word for hi'))
print('Using LLM', memory.retrieve_by_llm('Another word for hi'))

Using embeddings ['hello']
Using LLM ['hello']


In [12]:
# Visualise the keys in the database
database.keys()

dict_keys(['hello', 'Another word for hi', 'no worries', 'goodbye', 'hurray'])

In [13]:
print('Using embeddings', memory.retrieve_by_ranker('What to say when leaving'))
print('Using LLM', memory.retrieve_by_llm('What to say when leaving'))

Using embeddings ['goodbye']
Using LLM ['goodbye']


In [14]:
# Visualise the keys in the database
database.keys()

dict_keys(['hello', 'Another word for hi', 'no worries', 'goodbye', 'hurray', 'What to say when leaving'])